In [1]:
import re
import numpy as np

import findspark
import os
findspark.init()

import pyspark
sc = pyspark.SparkContext()


In [4]:
# this code will look for term through all pages of newspaper instead of only the front page 
# to search only the front page, need better tag parsing using lxml
# data for this is on VM ubuntu@206.12.96.6
# to access the notebook that is running already, set up tunnel
# ssh  ubuntu@206.12.96.6 -L 9995:localhost:9995
# then go to http://localhost:9995/tree in your browser

def process_string(whole_newspaper):
    list_of_lines=whole_newspaper.split("\n")
    searchTerm = "war"
    searchTerm = searchTerm.lower()
    regEx = re.compile(r"<wd pos=\"(\d+,\d+,\d+,\d+)\">([\w\'\",\[\]\{\}\(\)\.]+)</wd>")
    
    list_positions=[]
    for line in list_of_lines:
        line=line.strip()
        m = re.search(regEx,line)
        if m !=None :
            if m.group(2).lower() == searchTerm:
                list_positions.append(m.group(1))
    return list_positions
    
def see_set_content(something):
    return process_string(something[1]) #something[0] is the name of file, something[1] is the content


frontPage = np.zeros((7000,5250)) 
frontPage.dtype
frontPage = frontPage.astype('uint32')

# use wild cards to select range of dates
wholefiles = sc.wholeTextFiles('hdfs://hdp:9000/ocrdata/*1944*FEB10*')
wholefiles.cache()

output = wholefiles.map(see_set_content)

for l in output.collect():
    for j in l:
        pos = str(j).split(',')
        posList = [int(item) for item in pos]
        frontPage[posList[1]:posList[3],posList[0]:posList[2]] += 1
        
import matplotlib.pyplot as plt
% matplotlib inline
plt.figure(figsize=(200, 200))
image  = plt.imshow(frontPage)
plt.show()